In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import csv
import pandas as pd
from datetime import date
import math
from dateutil.relativedelta import relativedelta

In [21]:
DFcontracts = pd.read_csv('/content/drive/MyDrive/GIS_stuff/Contracts.csv')

In [18]:
DFcontracts

,contract_id,contract_startdate,contract_enddate,contract_total_fees,contract_deposit_fees,client_name,contract_payment_type,max_grace_period ( Max working days )
0,101,1/1/2021,1/1/2023,500000,NaN,Motaheda Co,ANNUAL,7
1,102,1/3/2021,1/3/2024,600000,10000.0,El Ahly Bank,QUARTER,5
2,103,1/5/2021,1/5/2023,400000,50000.0,Misr Pertol,QUARTER,14
3,104,1/3/2021,1/3/2024,700000,NaN,El Nagah Club,MONTHLY,3
4,105,1/4/2021,1/4/2026,900000,300000.0,OSR Co.,ANNUAL,14
5,106,1/1/2021,1/1/2026,1000000,200000.0,Masters Edu,HALF_ANNUAL,10


In [22]:
DFcontracts['contract_startdate'] = pd.to_datetime(DFcontracts['contract_startdate'])
DFcontracts['contract_enddate'] = pd.to_datetime(DFcontracts['contract_enddate'])
DFcontracts['contract_total_fees'] = pd.to_numeric(DFcontracts['contract_total_fees'], errors='coerce')
DFcontracts['contract_deposit_fees'] = pd.to_numeric(DFcontracts['contract_deposit_fees'], errors='coerce')
DFcontracts['contract_deposit_fees'] = DFcontracts['contract_deposit_fees'].fillna(0)
DFcontracts['max_grace_period'] = DFcontracts['max_grace_period ( Max working days )']
DFcontracts.drop('max_grace_period ( Max working days )', axis=1, inplace=True)

In [23]:
DFcontracts

,contract_id,contract_startdate,contract_enddate,contract_total_fees,contract_deposit_fees,client_name,contract_payment_type,max_grace_period
0,101,2021-01-01,2023-01-01,500000,0.0,Motaheda Co,ANNUAL,7
1,102,2021-01-03,2024-01-03,600000,10000.0,El Ahly Bank,QUARTER,5
2,103,2021-01-05,2023-01-05,400000,50000.0,Misr Pertol,QUARTER,14
3,104,2021-01-03,2024-01-03,700000,0.0,El Nagah Club,MONTHLY,3
4,105,2021-01-04,2026-01-04,900000,300000.0,OSR Co.,ANNUAL,14
5,106,2021-01-01,2026-01-01,1000000,200000.0,Masters Edu,HALF_ANNUAL,10


In [24]:
def get_payments_per_year(payment_type):
    if payment_type == "ANNUAL":
        return 1
    elif payment_type == "HALF_ANNUAL":
        return 2
    elif payment_type == "QUARTER":
        return 4
    elif payment_type == "MONTHLY":
        return 12
    else:
        return 1

for i in range(len(DFcontracts['contract_payment_type'])):
    payment_type = DFcontracts['contract_payment_type'][i]
    divisor = get_payments_per_year(payment_type)
    print(f'The number of installments to pay for the {i+1}th client is {divisor}')


The number of installments to pay for the 1th client is 1
The number of installments to pay for the 2th client is 4
The number of installments to pay for the 3th client is 4
The number of installments to pay for the 4th client is 12
The number of installments to pay for the 5th client is 1
The number of installments to pay for the 6th client is 2


In [25]:


def calculate_payments(row):
    start_date = pd.to_datetime(row['contract_startdate'])
    end_date = pd.to_datetime(row['contract_enddate'])
    payments_per_year = get_payments_per_year(row['contract_payment_type'])

    days_diff = (end_date - start_date).days
    years_diff = days_diff / 365.25

    total_payments = years_diff * payments_per_year

    return int(total_payments) if total_payments.is_integer() else int(total_payments) + 1


In [26]:
DFcontracts['No.ofPayments'] = DFcontracts.apply(calculate_payments, axis=1)

In [27]:
DFcontracts['No.ofPayments']

,No.ofPayments
0,2
1,12
2,8
3,36
4,5
5,10


In [28]:
def cal_payment_amount(row):
  ammount = (row['contract_total_fees'] - row['contract_deposit_fees']) / row['No.ofPayments']
  return ammount

In [29]:
DFcontracts['payment_ammount'] = DFcontracts.apply(cal_payment_amount, axis=1)

In [30]:
DFcontracts['payment_ammount']

,payment_ammount
0,250000.000000
1,49166.666667
2,43750.000000
3,19444.444444
4,120000.000000
5,80000.000000


In [31]:
def get_payment_deadline(row):
    start_date = pd.to_datetime(row['contract_startdate'])
    payments_per_year = get_payments_per_year(row['contract_payment_type'])
    grace_period_days = row.get('grace_period', 0)
    num_payments = row['No.ofPayments']

    months_per_period = 12 / payments_per_year
    deadlines = []

    for i in range(1, num_payments + 1):
        deadline = start_date + relativedelta(days=grace_period_days) + relativedelta(months=int((i - 1) * months_per_period))

        fractional_month = ((i - 1) * months_per_period) % 1
        if fractional_month:
            days_in_month = (deadline + relativedelta(months=1) - deadline).days
            deadline += relativedelta(days=int(fractional_month * days_in_month))

        deadlines.append(deadline)

    return deadlines


In [32]:
DFcontracts['first_payment_deadline'] = DFcontracts.apply(get_payment_deadline, axis=1)


In [33]:
DFcontracts['first_payment_deadline']

,first_payment_deadline
0,"[2021-01-01 00:00:00, 2022-01-01 00:00:00]"
1,"[2021-01-03 00:00:00, 2021-04-03 00:00:00, 202..."
2,"[2021-01-05 00:00:00, 2021-04-05 00:00:00, 202..."
3,"[2021-01-03 00:00:00, 2021-02-03 00:00:00, 202..."
4,"[2021-01-04 00:00:00, 2022-01-04 00:00:00, 202..."
5,"[2021-01-01 00:00:00, 2021-07-01 00:00:00, 202..."


In [34]:
DFcontracts.columns

Index(['contract_id', 'contract_startdate', 'contract_enddate',
       'contract_total_fees', 'contract_deposit_fees', 'client_name',
       'contract_payment_type', 'max_grace_period', 'No.ofPayments',
       'payment_ammount', 'first_payment_deadline'],
      dtype='object')

In [36]:
for index, row in DFcontracts.iterrows():
    contract_id = row['contract_id']
    contract_startdate = row['contract_startdate']
    contract_enddate = row['contract_enddate']
    contract_payment_type = row['contract_payment_type']
    num_payments = row['No.ofPayments']
    payment_amount = row['payment_ammount']
    client_name = row['client_name']

    print(f"Processing Contract ID: {contract_id} for {client_name}")
    print(f"  Start Date: {contract_startdate}")
    print(f"  End Date: {contract_enddate}")
    print(f"  Payment Type: {contract_payment_type}")
    print(f"  Number of Payments: {num_payments}")
    print(f"  Payment Amount: {payment_amount}")
    print("-" * 20)

Processing Contract ID: 101 for Motaheda Co
  Start Date: 2021-01-01 00:00:00
  End Date: 2023-01-01 00:00:00
  Payment Type: ANNUAL
  Number of Payments: 2
  Payment Amount: 250000.0
--------------------
Processing Contract ID: 102 for El Ahly Bank
  Start Date: 2021-01-03 00:00:00
  End Date: 2024-01-03 00:00:00
  Payment Type: QUARTER
  Number of Payments: 12
  Payment Amount: 49166.666666666664
--------------------
Processing Contract ID: 103 for Misr Pertol
  Start Date: 2021-01-05 00:00:00
  End Date: 2023-01-05 00:00:00
  Payment Type: QUARTER
  Number of Payments: 8
  Payment Amount: 43750.0
--------------------
Processing Contract ID: 104 for El Nagah Club
  Start Date: 2021-01-03 00:00:00
  End Date: 2024-01-03 00:00:00
  Payment Type: MONTHLY
  Number of Payments: 36
  Payment Amount: 19444.444444444445
--------------------
Processing Contract ID: 105 for OSR Co.
  Start Date: 2021-01-04 00:00:00
  End Date: 2026-01-04 00:00:00
  Payment Type: ANNUAL
  Number of Payments: 5


In [37]:
payment_schedules = {}

for index, row in DFcontracts.iterrows():
    contract_id = row['contract_id']
    contract_startdate = row['contract_startdate']
    contract_payment_type = row['contract_payment_type']
    num_payments = row['No.ofPayments']

    payments_per_year = get_payments_per_year(contract_payment_type)
    months_per_period = 12 / payments_per_year

    payment_dates = []
    for i in range(num_payments):
        payment_date = contract_startdate + relativedelta(months=int(i * months_per_period))
        payment_dates.append(payment_date)

    payment_schedules[contract_id] = payment_dates

for contract_id, dates in payment_schedules.items():
    print(f"Contract ID: {contract_id}")
    for date in dates:
        print(f"  Payment Date: {date.date()}")
    print("-" * 20)


Contract ID: 101
  Payment Date: 2021-01-01
  Payment Date: 2022-01-01
--------------------
Contract ID: 102
  Payment Date: 2021-01-03
  Payment Date: 2021-04-03
  Payment Date: 2021-07-03
  Payment Date: 2021-10-03
  Payment Date: 2022-01-03
  Payment Date: 2022-04-03
  Payment Date: 2022-07-03
  Payment Date: 2022-10-03
  Payment Date: 2023-01-03
  Payment Date: 2023-04-03
  Payment Date: 2023-07-03
  Payment Date: 2023-10-03
--------------------
Contract ID: 103
  Payment Date: 2021-01-05
  Payment Date: 2021-04-05
  Payment Date: 2021-07-05
  Payment Date: 2021-10-05
  Payment Date: 2022-01-05
  Payment Date: 2022-04-05
  Payment Date: 2022-07-05
  Payment Date: 2022-10-05
--------------------
Contract ID: 104
  Payment Date: 2021-01-03
  Payment Date: 2021-02-03
  Payment Date: 2021-03-03
  Payment Date: 2021-04-03
  Payment Date: 2021-05-03
  Payment Date: 2021-06-03
  Payment Date: 2021-07-03
  Payment Date: 2021-08-03
  Payment Date: 2021-09-03
  Payment Date: 2021-10-03
  Pay

In [38]:
payment_schedule_list = []

for index, row in DFcontracts.iterrows():
    contract_id = row['contract_id']
    client_name = row['client_name']
    payment_amount = row['payment_ammount']
    payment_dates = payment_schedules[contract_id]

    for i, payment_date in enumerate(payment_dates):
        payment_schedule_list.append({
            'client_name': client_name,
            'contract_id': contract_id,
            'payment_number': i + 1,
            'payment_date': payment_date,
            'payment_amount': payment_amount
        })

DFpayment_schedule = pd.DataFrame(payment_schedule_list)
display(DFpayment_schedule.head())

,client_name,contract_id,payment_number,payment_date,payment_amount
0,Motaheda Co,101,1,2021-01-01,250000.000000
1,Motaheda Co,101,2,2022-01-01,250000.000000
2,El Ahly Bank,102,1,2021-01-03,49166.666667
3,El Ahly Bank,102,2,2021-04-03,49166.666667
4,El Ahly Bank,102,3,2021-07-03,49166.666667


In [39]:
display(DFpayment_schedule)

,client_name,contract_id,payment_number,payment_date,payment_amount
0,Motaheda Co,101,1,2021-01-01,250000.000000
1,Motaheda Co,101,2,2022-01-01,250000.000000
2,El Ahly Bank,102,1,2021-01-03,49166.666667
3,El Ahly Bank,102,2,2021-04-03,49166.666667
4,El Ahly Bank,102,3,2021-07-03,49166.666667
...,...,...,...,...,...
68,Masters Edu,106,6,2023-07-01,80000.000000
69,Masters Edu,106,7,2024-01-01,80000.000000
70,Masters Edu,106,8,2024-07-01,80000.000000
71,Masters Edu,106,9,2025-01-01,80000.000000


In [40]:
grouped_payment_schedule = DFpayment_schedule.groupby('client_name')

In [41]:
for client_name, client_df in grouped_payment_schedule:
    filename = f"{client_name.replace(' ', '_')}.csv"
    client_df.to_csv(filename, index=False)
    print(f"Saved payment schedule for {client_name} to {filename}")

Saved payment schedule for El Ahly Bank to El_Ahly_Bank.csv
Saved payment schedule for El Nagah Club to El_Nagah_Club.csv
Saved payment schedule for Masters Edu to Masters_Edu.csv
Saved payment schedule for Misr Pertol to Misr_Pertol.csv
Saved payment schedule for Motaheda Co to Motaheda_Co.csv
Saved payment schedule for OSR Co. to OSR_Co..csv
